In [1]:
import pandas as pd
import numpy as np
import string
import re
import random
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer


In [2]:
df=pd.read_csv(r"D:\Capstone\Datasets\Data_preparation\preprocessed\Preprocessed_2000_samples.csv")


In [3]:
import ast

df['text_lemmatized'] = df['text_lemmatized'].apply(ast.literal_eval)

# Now you can proceed with concatenating the lists into sentences
df['sentences'] = df['text_lemmatized'].apply(lambda x: ' '.join(x))



In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorizer
vect = CountVectorizer()  
vects = vect.fit_transform(df.sentences)
vects.shape

(2000, 20782)

In [5]:
tdm = vects.T

term_document_matrix = pd.DataFrame.sparse.from_spmatrix(tdm, index=vect.get_feature_names_out(), columns=df['docid'].astype(str))

print(term_document_matrix.shape)

(20782, 2000)


## Max Normalization

In [6]:
import numpy as np

max_values = term_document_matrix.max(axis=0)
normalized_term_document_matrix =np.divide(term_document_matrix, max_values)
normalized_term_document_matrix=normalized_term_document_matrix.values


## TruncatedSVD

In [7]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

k = 200 
svd = TruncatedSVD(n_components=k)

U = svd.fit_transform(normalized_term_document_matrix)
S = np.diag(svd.singular_values_)  
VT = svd.components_


S_inv = np.linalg.inv(S)

print("Inverse of S:")
print(S_inv)

Inverse of S:
[[0.02287395 0.         0.         ... 0.         0.         0.        ]
 [0.         0.05431071 0.         ... 0.         0.         0.        ]
 [0.         0.         0.05737427 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.2704472  0.         0.        ]
 [0.         0.         0.         ... 0.         0.27097399 0.        ]
 [0.         0.         0.         ... 0.         0.         0.2726388 ]]


In [8]:
Xq_prime = normalized_term_document_matrix.T

print("Xq_prime shape: ",Xq_prime.shape)
print("U shape: ",U.shape)
print("S_inv shape: ",S_inv.shape)


Xq_prime shape:  (2000, 20782)
U shape:  (20782, 200)
S_inv shape:  (200, 200)


In [9]:
Dq = Xq_prime.dot(U).dot(S_inv)
Dq.shape

(2000, 200)

In [10]:
a=list(df['encoded_topics'].unique())
len(a)

10

In [11]:
class_mapping = {old_label: new_label for new_label, old_label in enumerate(df['encoded_topics'].unique())}
df['encoded_topics'] = df['encoded_topics'].map(class_mapping)
df['encoded_topics'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [12]:
reverse_class_mapping = {v: k for k, v in class_mapping.items()}
print("Old Class Categories:")
for encoded_label, old_label in reverse_class_mapping.items():
    print(f"Encoded Label {encoded_label}: Original Label {old_label}")


Old Class Categories:
Encoded Label 0: Original Label 69
Encoded Label 1: Original Label 4
Encoded Label 2: Original Label 41
Encoded Label 3: Original Label 20
Encoded Label 4: Original Label 55
Encoded Label 5: Original Label 0
Encoded Label 6: Original Label 27
Encoded Label 7: Original Label 6
Encoded Label 8: Original Label 13
Encoded Label 9: Original Label 52


## Model

## Cross Validation

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold

num_folds = 10

# Initialize cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize variables to store test accuracies
test_accuracies = []

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(skf.split(Dq, df['encoded_topics'])):
    print(f'Fold {fold + 1}/{num_folds}')

    # Split data into train and test sets for this fold
    X_train, X_test = Dq[train_index], Dq[test_index]
    y_train, y_test = df['encoded_topics'].iloc[train_index], df['encoded_topics'].iloc[test_index]

    num_classes = len(np.unique(np.concatenate((y_train, y_test))))

    y_train_encoded = to_categorical(y_train, num_classes=num_classes)
    y_test_encoded = to_categorical(y_test, num_classes=num_classes)

    X_train_flattened = X_train.reshape((X_train.shape[0], -1))
    X_test_flattened = X_test.reshape((X_test.shape[0], -1))

    # Define the model architecture
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train_flattened.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train_flattened, y_train_encoded, epochs=100, batch_size=64, 
                        validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test_flattened, y_test_encoded)
    print(f'Test accuracy for fold {fold + 1}: {test_accuracy*100:.2f}%')

    # Store the test accuracy for this fold
    test_accuracies.append(test_accuracy)

# Calculate and print the average test accuracy across all folds
avg_test_accuracy = np.mean(test_accuracies)
print(f'Average test accuracy: {avg_test_accuracy*100:.2f}%')

Fold 1/10
7/7 [==============================] - 0s 15ms/step - loss: 0.7407 - accuracy: 0.7700
Test accuracy for fold 1: 77.00%
Fold 2/10
7/7 [==============================] - 0s 21ms/step - loss: 0.7603 - accuracy: 0.7700
Test accuracy for fold 2: 77.00%
Fold 3/10
7/7 [==============================] - 0s 6ms/step - loss: 0.7679 - accuracy: 0.7700
Test accuracy for fold 3: 77.00%
Fold 4/10
7/7 [==============================] - 0s 4ms/step - loss: 0.8578 - accuracy: 0.7450
Test accuracy for fold 4: 74.50%
Fold 5/10
7/7 [==============================] - 0s 6ms/step - loss: 0.6966 - accuracy: 0.8000
Test accuracy for fold 5: 80.00%
Fold 6/10
7/7 [==============================] - 0s 67ms/step - loss: 0.7103 - accuracy: 0.7750
Test accuracy for fold 6: 77.50%
Fold 7/10
7/7 [==============================] - 0s 8ms/step - loss: 0.6630 - accuracy: 0.8000
Test accuracy for fold 7: 80.00%
Fold 8/10
7/7 [==============================] - 0s 4ms/step - loss: 0.5937 - accuracy: 0.8250
Test a

In [14]:
from sklearn.metrics import classification_report
from numpy import argmax, unique
import numpy as np


predictions = model.predict(X_test_flattened)  # Ensure this matches your reshaped test data variable


predictions_int = argmax(predictions, axis=1)


y_test_int = argmax(y_test_encoded, axis=1)


all_classes = unique(np.concatenate((y_test_int, predictions_int)))


target_names = [f'Class {i+1}' for i in all_classes]  


report = classification_report(y_test_int, predictions_int, target_names=target_names)

print(report)


7/7 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

     Class 1       0.83      1.00      0.91        20
     Class 2       0.85      0.85      0.85        20
     Class 3       0.88      0.70      0.78        20
     Class 4       0.86      0.95      0.90        20
     Class 5       0.68      0.65      0.67        20
     Class 6       0.74      0.70      0.72        20
     Class 7       0.88      0.75      0.81        20
     Class 8       0.71      0.75      0.73        20
     Class 9       0.75      0.90      0.82        20
    Class 10       1.00      0.90      0.95        20

    accuracy                           0.81       200
   macro avg       0.82      0.82      0.81       200
weighted avg       0.82      0.81      0.81       200



In [15]:
num_folds = 10

# Initialize cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize variables to store test accuracies
test_accuracies = []

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(skf.split(Dq, df['encoded_topics'])):
    print(f'Fold {fold + 1}/{num_folds}')

    # Split data into train and test sets for this fold
    X_train, X_test = Dq[train_index], Dq[test_index]
    y_train, y_test = df['encoded_topics'].iloc[train_index], df['encoded_topics'].iloc[test_index]

    num_classes = len(np.unique(np.concatenate((y_train, y_test))))

    y_train_encoded = to_categorical(y_train, num_classes=num_classes)
    y_test_encoded = to_categorical(y_test, num_classes=num_classes)

    X_train_flattened = X_train.reshape((X_train.shape[0], -1))
    X_test_flattened = X_test.reshape((X_test.shape[0], -1))

    # Define the model architecture
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train_flattened.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train_flattened, y_train_encoded, epochs=100, batch_size=64, 
                        validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test_flattened, y_test_encoded)
    print(f'Test accuracy for fold {fold + 1}: {test_accuracy*100:.2f}%')

    # Store the test accuracy for this fold
    test_accuracies.append(test_accuracy)

# Calculate and print the average test accuracy across all folds
avg_test_accuracy = np.mean(test_accuracies)
print(f'Average test accuracy: {avg_test_accuracy*100:.2f}%')

Fold 1/10
7/7 [==============================] - 0s 5ms/step - loss: 0.7884 - accuracy: 0.7350
Test accuracy for fold 1: 73.50%
Fold 2/10
7/7 [==============================] - 0s 6ms/step - loss: 0.7392 - accuracy: 0.7750
Test accuracy for fold 2: 77.50%
Fold 3/10
7/7 [==============================] - 0s 4ms/step - loss: 0.7210 - accuracy: 0.7600
Test accuracy for fold 3: 76.00%
Fold 4/10
7/7 [==============================] - 0s 41ms/step - loss: 0.8571 - accuracy: 0.7100
Test accuracy for fold 4: 71.00%
Fold 5/10
7/7 [==============================] - 0s 16ms/step - loss: 0.6709 - accuracy: 0.7850
Test accuracy for fold 5: 78.50%
Fold 6/10
7/7 [==============================] - 0s 2ms/step - loss: 0.7009 - accuracy: 0.7500
Test accuracy for fold 6: 75.00%
Fold 7/10
7/7 [==============================] - 0s 2ms/step - loss: 0.6626 - accuracy: 0.7950
Test accuracy for fold 7: 79.50%
Fold 8/10
7/7 [==============================] - 0s 2ms/step - loss: 0.6168 - accuracy: 0.8200
Test ac

In [16]:
from sklearn.metrics import classification_report
from numpy import argmax, unique
import numpy as np


predictions = model.predict(X_test_flattened)  # Ensure this matches your reshaped test data variable


predictions_int = argmax(predictions, axis=1)


y_test_int = argmax(y_test_encoded, axis=1)


all_classes = unique(np.concatenate((y_test_int, predictions_int)))


target_names = [f'Class {i+1}' for i in all_classes]  


report = classification_report(y_test_int, predictions_int, target_names=target_names)

print(report)


7/7 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

     Class 1       0.87      1.00      0.93        20
     Class 2       0.88      0.75      0.81        20
     Class 3       0.92      0.60      0.73        20
     Class 4       0.81      0.85      0.83        20
     Class 5       0.70      0.80      0.74        20
     Class 6       0.65      0.65      0.65        20
     Class 7       0.88      0.75      0.81        20
     Class 8       0.76      0.80      0.78        20
     Class 9       0.70      0.95      0.81        20
    Class 10       1.00      0.90      0.95        20

    accuracy                           0.81       200
   macro avg       0.82      0.81      0.80       200
weighted avg       0.82      0.81      0.80       200



In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping



# Then proceed with train-test split
X_train, X_test, y_train, y_test = train_test_split(
    Dq, 
    df['encoded_topics'], 
    test_size=0.2, 
    random_state=42,
    stratify=df['encoded_topics']  # This ensures stratified splitting
)


num_classes = len(np.unique(y_train))

y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)



X_train_flattened = X_train.reshape((X_train.shape[0], -1))
X_test_flattened = X_test.reshape((X_test.shape[0], -1))

# Define the model architecture
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_flattened.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_flattened, y_train_encoded, epochs=100, batch_size=64, 
                    validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_flattened, y_test_encoded)
print(f'Test accuracy: {test_accuracy*100:.2f}%')


Epoch 1/100
20/20 [==============================] - 1s 17ms/step - loss: 2.2234 - accuracy: 0.2305 - val_loss: 2.0996 - val_accuracy: 0.4969
Epoch 2/100
20/20 [==============================] - 0s 8ms/step - loss: 1.9124 - accuracy: 0.5117 - val_loss: 1.6828 - val_accuracy: 0.6469
Epoch 3/100
20/20 [==============================] - 0s 8ms/step - loss: 1.4353 - accuracy: 0.6414 - val_loss: 1.2367 - val_accuracy: 0.6812
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 1.0233 - accuracy: 0.7039 - val_loss: 0.9694 - val_accuracy: 0.7094
Epoch 5/100
20/20 [==============================] - 0s 8ms/step - loss: 0.7621 - accuracy: 0.7734 - val_loss: 0.8511 - val_accuracy: 0.7094
Epoch 6/100
20/20 [==============================] - 0s 13ms/step - loss: 0.6583 - accuracy: 0.7930 - val_loss: 0.7940 - val_accuracy: 0.7312
Epoch 7/100
20/20 [==============================] - 0s 9ms/step - loss: 0.5559 - accuracy: 0.8383 - val_loss: 0.8073 - val_accuracy: 0.7312
Epoch 8/100

In [18]:
from sklearn.metrics import classification_report
from numpy import argmax, unique
import numpy as np


predictions = model.predict(X_test_flattened)  # Ensure this matches your reshaped test data variable


predictions_int = argmax(predictions, axis=1)


y_test_int = argmax(y_test_encoded, axis=1)


all_classes = unique(np.concatenate((y_test_int, predictions_int)))


target_names = [f'Class {i+1}' for i in all_classes]  


report = classification_report(y_test_int, predictions_int, target_names=target_names)

print(report)


13/13 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

     Class 1       0.84      0.95      0.89        40
     Class 2       0.80      0.60      0.69        40
     Class 3       1.00      0.75      0.86        40
     Class 4       0.76      0.78      0.77        40
     Class 5       0.62      0.70      0.66        40
     Class 6       0.56      0.50      0.53        40
     Class 7       0.75      0.68      0.71        40
     Class 8       0.64      0.68      0.66        40
     Class 9       0.66      0.88      0.75        40
    Class 10       0.88      0.93      0.90        40

    accuracy                           0.74       400
   macro avg       0.75      0.74      0.74       400
weighted avg       0.75      0.74      0.74       400

